# Units in Python

In [ ]:
import numpy as np

### Find the position (x) of a rocket moving at a constant velocity (v) after a time (t)

<img src="https://uwashington-astro300.github.io/A300_images/rocket.png" width="400"/>

In [ ]:
def find_position(velocity, time):
    result = velocity * time
    return result

### If v = 10 m/s and t = 10 s

In [ ]:
my_velocity = 10
my_time = 10

find_position(my_velocity, my_time)

### No problem, x = 100 m

---

### Now v = 10 mph and t = 10 minutes

In [ ]:
my_other_velocity = 10
my_other_time = 10

find_position(my_other_velocity, my_other_time)

### x = 100 miles minutes / hour ??

---
# The Astropy Units package to the rescue

In [ ]:
from astropy import units as u
from astropy import constants as const
from astropy.units import imperial
imperial.enable()

#### *Note: because we imported the `units` package as `u`, you cannot use **u** as a variable name.*

---

### Add units to values using `u.UNIT` where UNIT is an [Astropy Unit](http://docs.astropy.org/en/stable/units/index.html#module-astropy.units.si)

* To add a UNIT to a VALUE you multiply (*) the VALUE by the UNIT
* You can make compound units like: `u.m / u.s`
* Use `()` to make your compound units easier to read

In [ ]:
my_velocity = 10 * (u.m / u.s)
my_time = 10 * u.s

In [ ]:
def find_position(velocity, time):
    result = velocity * time
    return result

In [ ]:
find_position(my_velocity, my_time)

#### Notice the difference when using imperial units - (`imperial.UNIT`)

In [ ]:
my_other_velocity = 10.0 * (imperial.mi / u.h)
my_other_time = 10 * u.min

In [ ]:
find_position(my_other_velocity, my_other_time)

### Notice that the units are a bit strange. We can simplify this using `.decompose()`

* Default to SI units

In [ ]:
find_position(my_other_velocity, my_other_time).decompose()

### I like to put the `.decompose()` in the return of the function:

In [ ]:
def find_position(velocity, time):
    result = velocity * time
    return result.decompose()

In [ ]:
find_position(my_other_velocity, my_other_time)

### The really great thing about the `units` package is that you can use any unit as long as it is the correct type.

- For example, the `find_position` function will work as long as the velocity is (length / time) and time is (time)
  - Velocity = 1 furlong / fortnight
  - Time = 1 week

In [ ]:
find_position(1 * (imperial.fur / u.fortnight), 1 * u.wk)

### Unit conversion is really easy!

-  `.to(u.UNIT)`

In [ ]:
find_position(my_other_velocity, my_other_time)

In [ ]:
find_position(my_other_velocity, my_other_time).to(u.km)

In [ ]:
find_position(my_other_velocity, my_other_time).to(imperial.mi)

In [ ]:
find_position(my_other_velocity, my_other_time).si                     # quick conversion to SI units

In [ ]:
find_position(my_other_velocity, my_other_time).cgs                    # quick conversion to CGS units

## It is always better to do unit conversions **outside** of functions

----
## Unit equivalencies

In many fields there are very common unit conversions that do not seem to make sense based on the units involved.

For example in astronomy, the distances to stars can be based on the parallax angle. Parallax is an angle and distance is a length, 

$$ \Large
\mathrm{Distance\ in\ pc}\ (d_{\mathrm{pc}}) \approx \frac{1}{p\,^{\prime\prime}}
$$

If you do not know the context of the convesion, it does not seem to be correct.

The Astropy units package has a command called `equivalencies` that allow you to do unit conversions under certain physical assumptions.


### Parallax -> Distance  `equivalencies=u.parallax()`

In [ ]:
my_parallax = 34.26 * u.mas

my_parallax

In [ ]:
# This will not work!

my_parallax.to(u.parsec)

### With `equivalencies` we can convert it to **any** length

In [ ]:
my_parallax.to(u.parsec, equivalencies=u.parallax())

In [ ]:
my_parallax.to(u.lyr, equivalencies=u.parallax())

In [ ]:
my_parallax.to(u.AU, equivalencies=u.parallax())

### You can find the units in `Astropy` that are of the same type with `.find_equivalent_units()`

In [ ]:
(u.pc).find_equivalent_units()

### Another common astronomy example is in spectroscopy (light).

### Wavelength -> Frequency or Energy `equivalencies=u.spectral()`

In [ ]:
my_wavelength = 380 * u.nm

my_wavelength

In [ ]:
my_wavelength.to(u.Hz, equivalencies=u.spectral())

In [ ]:
my_wavelength.to(u.eV, equivalencies=u.spectral())

---
### You do not have to worry about working in different units (**as long as they are the same type**)!

* No conversions needed
* Just make sure you assign units

In [ ]:
my_velocity

In [ ]:
my_other_velocity

In [ ]:
my_velocity + my_other_velocity

#### Units default to SI units

In [ ]:
my_time, my_other_time

In [ ]:
my_time + my_other_time

---
### Be careful combining quantities with different units!

In [ ]:
my_velocity + my_time

In [ ]:
2 + my_time

---

## Using units can save you headaches. 

* All of the trig functions expect all angles to be in radians. 
* If you forget this, it can lead to problems that are hard to debug

$$ \large
\sin(90^{\circ}) + \sin(45^{\circ}) =  1 + \frac{\sqrt{2}}{2} \approx 1.7071
$$

In [ ]:
np.sin(90) + np.sin(45)

In [ ]:
np.sin(90 * u.deg) + np.sin(45 * u.deg)

---

## You can define your own units

In [ ]:
ringo = u.def_unit('Ringos', 3.712 * imperial.yd)

In [ ]:
find_position(my_velocity, my_time).to(ringo)

In [ ]:
my_velocity.to(ringo / u.s)

#### ...Since `ringo` is self-defined it does not have a `u.` in front of it

---

# Constants

The `Astropy` package also includes a whole bunch of built-in constants to make your life easier.

* The package is usually imported as `const`

## [Astropy Constants](http://docs.astropy.org/en/stable/constants/index.html#reference-api)

In [ ]:
const.G

In [ ]:
const.M_sun

---

### An Example: The velocity of an object in circular orbit around the Sun is

$$\Large
v=\sqrt{GM_{\odot}\over d} 
$$

### What is the velocity of an object at 1 AU from the Sun?

In [ ]:
def find_orbit_v(my_distance):
    result = np.sqrt(const.G * const.M_sun / my_distance)
    return result.decompose()

In [ ]:
find_orbit_v(my_distance = 1 * u.AU)

In [ ]:
find_orbit_v(my_distance = 1 * u.AU).to(u.km / u.s)

In [ ]:
find_orbit_v(my_distance = 1 * u.AU).to(ringo / u.ms)

## Again, as long as your distance has *any* units of `length` this will work!

In [ ]:
find_orbit_v(my_distance = 1 * u.lyr)